In [1]:
# let the notebook access the code folder
import sys
sys.path.insert(1,"code")

## testing exp2bytecode parser

In [5]:
from exp2bytecode_lex import lexer, tokens

In [6]:
from exp2bytecode_gram import parser

In [7]:
program1 = \
'''
      store x 3;
      store y 2;
      pushv y;	
      pushv x; 
      call add;	
      popv;		
      popv;		
      print "The sum x+y is " %rvx;
      stop;

add:
     store %rvx (+ %tsx[-1] %tsx[-2]);
     return;
'''

In [8]:
parser.parse(program1)

In [9]:
program2 = \
'''
      store x 3;
      store y 2;
      pushv y;		# push second parameter onto stack
      pushv x;		# push first parameter onto stack
      call add;		# push current address onto stack and jump to function
      popv;		    # pop first parameter
      popv;		    # pop second parameter
      print "The sum x+y is " %rvx;
      stop;

add:
     store a %tsx[-1];	    # get value of first parameter
     store b %tsx[-2];	    # get value of second parameter
     store %rvx (+ a b);	# store the sum in the ‘return value register’
     return;		        # pop return address off stack and return to that address
'''

In [10]:
parser.parse(program2)

In [11]:
program3 = \
'''
     # print the sequence 0 1 2 using recursion
     pushv 2;
     call seq;
     popv;
     stop;

seq:
     pushf 1;                 # push a frame of size 1 - variable x
     store %tsx[0] %tsx[-2];  # initialize x with the actual parameter
     jumpF %tsx[0] L1;        # test x
     pushv (- %tsx[0] 1);       
     call seq;
     popv;
L1:
     print %tsx[0];
     popf 1;                  # remove the frame from the stack
     return;

'''

In [12]:
parser.parse(program3)

## testing exp2bytecode frontend

In [13]:
from exp2bytecode_lex import lexer, tokens

In [14]:
from exp2bytecode_interp_gram import parser

In [15]:
from pprint import pprint

In [16]:
from exp2bytecode_interp_state import state
state.initialize()

In [17]:
parser.parse(program1)

In [18]:
pprint(state.program)

[('store', ('id', 'x'), ('number', 3)),
 ('store', ('id', 'y'), ('number', 2)),
 ('pushv', ('id', 'y')),
 ('pushv', ('id', 'x')),
 ('call', 'add'),
 ('popv', None),
 ('popv', None),
 ('print', 'The sum x+y is ', ('%rvx',)),
 ('stop',),
 ('store',
  ('%rvx',),
  ('+',
   ('%tsx', ('UMINUS', ('number', 1))),
   ('%tsx', ('UMINUS', ('number', 2))))),
 ('return',)]


In [19]:
print(state.label_table)

{'add': 9}


## testing the abstract machine

In [20]:
from exp2bytecode_interp import interp


In [21]:
interp(program1)

The sum x+y is 5


In [24]:
interp(program2)

The sum x+y is 5


In [25]:
interp(program3)

> 0
> 1
> 2
